In [13]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

import time

from hist import Hist

import babar_analysis_tools as bat
from analysis_variables import *
import myPIDselector

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

import joblib
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
print(np.__version__)

print(plt.__version__)

1.26.4
1.26.4


In [3]:
BNC_tag = ""
BNC_bool = False
UNBLINDED_bool = True
if UNBLINDED_bool is True:
    BNC_tag = '_UNBLINDED'

# BNC
#BNC_tag = "_BNC"
#BNC_bool = True

#####################################################################
# Where are we running this?
#####################################################################
## Bellis computer
topdir= "/home/bellis/babar_data_local/bnv_plambda"

if BNC_bool:
    topdir= "/home/bellis/babar_data/bnv_plambda_bnc"

## My laptop
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
#####################################################################


#####################################################################
# Get the BNV data
#####################################################################
#data, data_collision = bat.load_datasets(topdir=topdir, subset='Run1')
data, data_collision = bat.load_datasets(topdir=topdir, subset='all', UNBLINDED=UNBLINDED_bool)

Opening /home/bellis/babar_data_local/bnv_plambda/Background_and_signal_SP_modes_All_runs.parquet...
Took 8.133 seconds

Opening /home/bellis/babar_data_local/bnv_plambda/Data_All_runs_UNBLINDED.parquet...
Took 1.910 seconds



In [4]:
dataset_information= pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

sp= data["spmode"]

splist= np.unique(sp.to_list())
splist

'''
dcuts= bat.get_final_masks(data, region_definitions= region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')

dcuts[3]
'''

bat.fill_new_entry_with_tag_side_B(data)
data["BtagSideMes"]
bat.fill_new_entry_with_tag_side_B(data_collision)
data_collision["BtagSideMes"]

all_hists= bat.create_empty_histograms(hist_defs)

bkg_spmodes= ["998","1005","3981","1235","1237"]
sig_spmodes= ["-999"]

spmodes= bkg_spmodes+sig_spmodes

weights= {}
for sp in spmodes: 
    weights[sp]= bat.scaling_value(int(sp),dataset_information=dataset_information, cs_data= cs_data, plot= False, verbose= False)
    #weights[sp]=1

weights["-999"]= 1000 #scales signal higher 
weights["0"]= 1 #idk what this is for;;; ASK

print(weights)

### ASK WHAT THESE MEAN
#tag= "EARLY_CUT"
cuts_tag= "CUTS_cuts_1_2_3"


/home/bellis/.local/lib/python3.11/site-packages/awkward/_nplikes/array_module.py:292: RuntimeWarning: invalid value encountered in sqrt
  return impl(*broadcasted_args, **(kwargs or {}))


{'998': 0.2506879075487834, '1005': 0.49619965664110677, '3981': 0.7950555869080849, '1235': 0.3191592629119508, '1237': 0.31492522877218293, '-999': 1000, '0': 1}


# Generate cuts

In [5]:

##################################################
# SP
dcuts= bat.get_final_masks(data, region_definitions= region_definitions)



###################################
# Collision
dcuts_col= bat.get_final_masks(data_collision, region_definitions= region_definitions)



In [6]:
subset = ['spmode', 'BpostFitMes', 'BpostFitDeltaE', 'Lambda0_unc_Mass', \
      'BtagSideMes', 'BSphr', 'BThrust', 'BCosThetaS', \
      'R2', 'R2All', \
      'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
      'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
      'BThrustROE', 'BcosthCM']


In [7]:
###################################
# SP
mask_event= dcuts[1]["event"]# & dcuts[2]["event"] & dcuts[3]["event"] 
mask= mask_event

df_sp = bat.dump_awkward_to_dataframe(data[mask], fields_to_dump=subset)#, dropna=True)

# Put the cuts into the dataframe 
cut1 = dcuts[1]['event'] # This is the main cut that gets rid of duplicates
cuts_to_add = [2, 3, 4, 6, -1]
for cut in cuts_to_add:
    bools = dcuts[cut]['event']
    colname = f'cut_{cut}'
    print(colname, len(bools[cut1]), bools[cut1])

    df_sp[colname] = bools[cut1]

# Drop entries where there are nans
print(len(df_sp))
df_sp.dropna(inplace=True)
print(len(df_sp))


cut_2 1347625 [ True False False ...  True  True  True]
cut_3 1347625 [ True  True False ...  True  True False]
cut_4 1347625 [False False False ...  True False  True]
cut_6 1347625 [False False  True ...  True  True False]
cut_-1 1347625 [False False False ...  True False False]
1347625
1345183


In [8]:
###################################################
# Collision
mask_event= dcuts_col[1]["event"]# & dcuts[2]["event"] & dcuts[3]["event"] 
mask= mask_event

df_col = bat.dump_awkward_to_dataframe(data_collision[mask], fields_to_dump=subset)

# Put the cuts into the dataframe 
cut1 = dcuts_col[1]['event'] # This is the main cut that gets rid of duplicates
cuts_to_add = [2, 3, 4, 6, -1]
for cut in cuts_to_add:
    bools = dcuts_col[cut]['event']
    colname = f'cut_{cut}'
    print(colname, len(bools[cut1]), bools[cut1])

    df_col[colname] = bools[cut1]

# Drop entries where there are nans
print(len(df_col))
df_col.dropna(inplace=True)
print(len(df_col))

cut_2 725839 [False False False ...  True  True False]
cut_3 725839 [ True  True  True ... False  True  True]
cut_4 725839 [False False False ... False False False]
cut_6 725839 [False  True  True ...  True  True  True]
cut_-1 725839 [False False False ... False False False]
725839
724129


In [15]:
ntrain_sig = 30000
ntrain_bkg = 30000
trial = 15
features = 4

ntrain_tag = f'features_{features}_nsig_{ntrain_sig}_nbkg_{ntrain_bkg}_trial{trial:0d}'

print(ntrain_tag)

# NO BNC tag in the name
model_filename = f"MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}_.pkl"

print(model_filename)
print()

#
if os.path.exists(model_filename):
    print(f"The file '{model_filename}' exists.")
else:
    print(f"The file '{model_filename}' does not exist.")

features_4_nsig_30000_nbkg_30000_trial15
MODEL_MLPClassifier_CUTS_1_2_3_features_4_nsig_30000_nbkg_30000_trial15_.pkl

The file 'MODEL_MLPClassifier_CUTS_1_2_3_features_4_nsig_30000_nbkg_30000_trial15_.pkl' exists.


In [18]:
# Load in the model
workspace = joblib.load(model_filename)

bat.add_probas_to_dfs(workspace, df_col, df_sp)#.drop(columns=columns_to_drop))


Training vars initially: 
Index(['BCosThetaS', 'R2All', 'thrustMagAll', 'BCosThetaT', 'BLegendreP2',
       'BcosthCM'],
      dtype='object')
6
Checking for proba...
Checking for used_in_sig_train...
Checking for used_in_bkg_train...
Training vars finally: 
Index(['BCosThetaS', 'R2All', 'thrustMagAll', 'BCosThetaT', 'BLegendreP2',
       'BcosthCM'],
      dtype='object')
6
[ 0.17588651  0.5814954  -0.2583879  ...  0.24225366 -0.6445146
 -0.4060091 ]


/home/bellis/micromamba/envs/pyhep/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/bellis/micromamba/envs/pyhep/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


0

In [20]:
print(f'{ntrain_tag}{BNC_tag}')

features_4_nsig_30000_nbkg_30000_trial15_UNBLINDED


In [21]:
outfilename = f"DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
df_sp.to_parquet(outfilename)

outfilename = f"DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
df_col.to_parquet(outfilename)

# Read in the files and plot